# Add Forecasts to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](../../notebook/PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add forecasts to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath,"test", "PowerSystems_examples", "parse_matpower.jl"))

Project SIIPExamples v0.0.1
    Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [9961bab8] Cbc v0.6.6
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.20.0
  [2cd47ed4] InfrastructureSystems v0.5.11
  [b6b21f68] Ipopt v0.6.1
  [2535ab7d] JSON2 v0.3.1
  [4076af6c] JuMP v0.21.1
  [98b081ad] Literate v2.3.0
  [47be7bcc] ORCA v0.3.1
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v0.29.6
  [5f7eddb3] PowerGraphics v0.2.0 #master (https://github.com/NREL-SIIP/PowerGraphics.jl.git)
  [e690365d] PowerSimulations v0.2.6
  [bcd98974] PowerSystems v0.12.0
  [9e3dc215] TimeSeries v0.16.0
  [0f1e0344] WebIO v0.8.13
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 
┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Zvssm/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbar

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
4,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
6,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
8,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir,"forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR,"timeseries_pointers_da.json")
open(fname,"r") do f
    for line in eachline(f)
        println(line)
    end
end

[
    {
        "simulation": "DAY_AHEAD",
        "category": "Generator",
        "component_name": "SolarBusC",
        "label": "get_rating",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/PV/da_solar5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "Generator",
        "component_name": "WindBusA",
        "label": "get_rating",
        "scaling_factor": 1.0,
        "data_file": "./gen/Renewable/WIND/da_wind5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "ElectricLoad",
        "component_name": "bus2",
        "label": "get_maxactivepower",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category": "ElectricLoad",
        "component_name": "bus3",
        "label": "get_maxactivepower",
        "scaling_factor": 1.0,
        "data_file": "./load/da_load5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "category":

### Read the pointers

In [3]:
ts_pointers = IS.read_time_series_metadata(fname)

5-element Array{InfrastructureSystems.TimeseriesFileMetadata,1}:
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "Generator", "SolarBusC", "get_rating", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/Zvssm/data/forecasts/5bus_ts/gen/Renewable/PV/da_solar5.csv", Float64[], "DeterministicInternal", nothing)
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "Generator", "WindBusA", "get_rating", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/Zvssm/data/forecasts/5bus_ts/gen/Renewable/WIND/da_wind5.csv", Float64[], "DeterministicInternal", nothing)
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "ElectricLoad", "bus2", "get_maxactivepower", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/Zvssm/data/forecasts/5bus_ts/load/da_load5.csv", Float64[], "DeterministicInternal", nothing)       
 InfrastructureSystems.TimeseriesFileMetadata("DAY_AHEAD", "ElectricLoad", "bus3", "get_maxactivepower", 1.0, "/Users/cbarrows/.julia/packages/PowerSystems/Zvssm/da

### Read and assign forecasts to `System` using the `ts_pointers` struct

In [4]:
add_forecasts!(sys, ts_pointers)
sys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
3,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
4,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
6,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
7,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,2
8,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*